In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns

## Loading train and test data sets

In [ ]:
df_train =pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.describe()

Now let's check test data.

In [ ]:
df_test=pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
df_test.head()

In [ ]:
df_test.isnull().sum()


In [ ]:
df_train.shape

## EDA



Some columns have little negative correlation with target variable.

These can prove to be important features to predict target.

In [ ]:
df_train.columns

## Split data for training & validation

Data is clean with continues target variable. let's make split into 70:20 train and validation data. and drop id column. 

Storing values in numpy array saves memory.

In [ ]:
x = df_train.iloc[:, 1:15].values  
print(x) 
y = df_train.iloc[:, -1].values 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)

In [ ]:
# Feature Scaling
#from sklearn.preprocessing import StandardScaler

#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

### LGB regressor

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import model_selection

In [ ]:
#from xgboost import XGBRegressor
import lightgbm as ltb

### RandomizedSearch: Hyperparameters

In [ ]:

#ltb_r = ltb.LGBMRegressor()

lgbm = ltb.LGBMRegressor()

In [ ]:
#Defining a dictionary containing all the releveant parameters
param_grid = {
    "boosting_type": ['gbdt'],
    "num_leaves": [ 19, 31, 37, 47],
    "max_depth": [7, 15, 29, 37, 47, 53], 
    "learning_rate": [0.1, 0.15, 0.01],
    "n_estimators": [500, 1000, 2000], 
    "subsample_for_bin": [20000, 200000, 2000000], 
    "objective": ["regression"],
    "min_child_weight": [0.001, 0.01], 
    "min_child_samples":[20, 50, 100], 
    "subsample":[1.0], 
    "subsample_freq":[0], 
    "colsample_bytree":[1.0], 
    "reg_alpha":[0.0], 
    "reg_lambda":[0.0]
}

In [ ]:
model = model_selection.RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_grid,
    n_iter=100,
    scoring="neg_root_mean_squared_error",
    verbose=10,
    n_jobs=-1,
    cv=5
)

In [ ]:
# fit the model and extract best score
model.fit(X_train, y_train)

In [ ]:
print(f"Best score: {model.best_score_}")
print("Best parameters from the RandomSearchCV:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print(f"\t{param_name}: {best_parameters[param_name]}")

In [ ]:

#ltb_r = ltb.LGBMRegressor()

'''
ltb_r = ltb.LGBMRegressor(boosting_type= 'gbdt', #'rf', #'goss',#'dart', #
                         num_leaves=31, 
                         max_depth= 11, #12, #16, #- 1, 
                         learning_rate=0.1, 
                         n_estimators=1000, #500, 
                         subsample_for_bin=200000, 
                         objective=None, 
                         class_weight=None, 
                         min_split_gain=0.0, 
                         min_child_weight=0.001, 
                         min_child_samples=20, 
                         subsample=1.0, 
                         subsample_freq=0, 
                         colsample_bytree=1.0, 
                         reg_alpha=0.0, 
                         reg_lambda=0.0, 
                         random_state=None, 
                         n_jobs=- 1, 
                         silent=True
                        )

ltb_r.fit(X_train,y_train)

'''

In [ ]:
# Get best model
best_model = model.best_estimator_

In [ ]:
y_pred= best_model.predict(X_train)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

For previous grid_search
    
    Mean Absolute Error: 0.590135219891411
    Mean Squared Error: 0.4941394332980038
    Root Mean Squared Error: 0.7029505198077627

In [ ]:
#X_t = sc.transform(df_test[:,1:])

## Prepare predictions for submission.csv

In [ ]:
# Make prediction for submission using best_estimators from grid search.
preds = best_model.predict(df_test.iloc[:,1:].values)
#preds=XGB.predict(df_test.iloc[:,1:].values)

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
sub.head()

In [ ]:
sub.target =preds
sub.to_csv("submission.csv", index=False)

In [ ]:
sub.head()